In [1]:
import pandas as pd

In [8]:
from bs4 import BeautifulSoup
import requests
import random

In [3]:
games = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')

games['GAME_DATE'] = games['GAME_DATE'].dt.strftime('%Y-%m-%d')

dates = games[games['GAME_DATE'] > '2019-10-01'].sort_values('GAME_DATE', ascending=False)['GAME_DATE'].unique().tolist()

In [4]:
date = '2020-07-08'

In [11]:
class HostHeaderSSLAdapter(requests.adapters.HTTPAdapter):
    def resolve(self, hostname):
        # a dummy DNS resolver
        import random
        ips = [
            '1.1.1.1',  # CloudFlare
            '8.8.8.8',  # Fastly
        ]
        resolutions = {
            'cdn.jsdelivr.net': random.choice(ips),
        }
        return resolutions.get(hostname)

    def send(self, request, **kwargs):
        from urllib.parse import urlparse

        connection_pool_kwargs = self.poolmanager.connection_pool_kw

        result = urlparse(request.url)
        resolved_ip = self.resolve(result.hostname)

        if result.scheme == 'https' and resolved_ip:
            request.url = request.url.replace(
                'https://' + result.hostname,
                'https://' + resolved_ip,
            )
            connection_pool_kwargs['server_hostname'] = result.hostname  # SNI
            connection_pool_kwargs['assert_hostname'] = result.hostname

            # overwrite the host header
            request.headers['Host'] = result.hostname
        else:
            # theses headers from a previous request may have been left
            connection_pool_kwargs.pop('server_hostname', None)
            connection_pool_kwargs.pop('assert_hostname', None)

        return super(HostHeaderSSLAdapter, self).send(request, **kwargs)


# url = 'https://cdn.jsdelivr.net/npm/bootstrap/LICENSE'

# session = requests.Session()
# session.mount('https://', HostHeaderSSLAdapter())

# r = session.get(url)
# print(r.headers)

# r = session.get(url)
# print(r.headers)

In [12]:
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
url = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={date}"
for i in range(len(user_agent_list)):
    try:
        #Pick a random user agent
        user_agent = random.choice(user_agent_list)
        #Set the headers 
        headers = {'User-Agent': user_agent}
        #Make the request
#         response = requests.get(url,headers=headers)
        session = requests.Session()
        session.mount('https://', HostHeaderSSLAdapter())

        r = session.get(url,headers=headers)
        print(r.headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        print("Request #%d\nUser-Agent Sent:%s\n\nHeaders Recevied by HTTPBin:"%(i,user_agent))
        print(soup)
        print("-------------------")
        break

    except Exception as e:
        print(e)

HTTPSConnectionPool(host='www.sportsbookreview.com', port=443): Max retries exceeded with url: /betting-odds/nba-basketball/?date=2020-07-08 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x117a277c0>: Failed to establish a new connection: [Errno 61] Connection refused'))
HTTPSConnectionPool(host='www.sportsbookreview.com', port=443): Max retries exceeded with url: /betting-odds/nba-basketball/?date=2020-07-08 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1187b42b0>: Failed to establish a new connection: [Errno 61] Connection refused'))
HTTPSConnectionPool(host='www.sportsbookreview.com', port=443): Max retries exceeded with url: /betting-odds/nba-basketball/?date=2020-07-08 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1187b4490>: Failed to establish a new connection: [Errno 61] Connection refused'))
HTTPSConnectionPool(host='www.sportsbookreview.com', port=443): Max retries exceeded with u

In [7]:
# URL = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={date}"
# headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}
# page = requests.get(url=URL, headers=headers)
# soup = BeautifulSoup(page.content, 'html.parser')

ConnectionError: HTTPSConnectionPool(host='www.sportsbookreview.com', port=443): Max retries exceeded with url: /betting-odds/nba-basketball/?date=2020-07-08 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x117a27580>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
teams = soup.find_all(class_='GameRows_participantBox__0WCRz')
spread = soup.find_all(class_="OddsCells_compact__cawia border-left")
wager_percentage = soup.find_all("span", class_="opener")
scores = soup.find_all(class_ = 'GameRows_scores__YkN24') 
opening_spread = soup.find_all(class_='GameRows_adjust__NZn2m GameRows_opener__NivKJ')

In [ ]:
temp = []
for j in range(int(7*len(teams)/2)):
    for w in [1, 2, 4, 5]:
        temp.append(spread[j].find_all("span")[w].text)

In [ ]:
spreads = []
odds = []
for i in range(len(temp)):
    if i % 2 == 0:
        spreads.append(temp[i])
    else:
        odds.append(temp[i])

In [ ]:
away = []
home = []
for i in range(len(spreads)):
    if i % 2 == 0:
        away.append(spreads[i])
        away.append(odds[i])
    else:
        home.append(spreads[i])
        home.append(odds[i])

In [ ]:
rows_to_add= []
temp_away = []
temp_home = []
splits = [i for i in range(13, (14*20), 14)]
for i in range(len(away)):
    temp_away.append(away[i])
    temp_home.append(home[i])
    if i in splits:
        rows_to_add.append(temp_away)
        rows_to_add.append(temp_home)
        temp_away = []
        temp_home = []

In [ ]:
row = []
for i in range(len(teams)):
    if i % 2 == 0:
        opponent = teams[i+1].text
        home = 0
    else:
        opponent = teams[i-1].text
        home = 1
    row.append([date, teams[i].text, home, opponent, scores[i].text, wager_percentage[i].text, opening_spread[i].text, '-110'])

In [ ]:
df_1 = pd.DataFrame(row, columns=['Game_Date', 'Team_Name', 'Home', 'Opponent', 'Points', 'Pct_of_Bets', 'Opening_Spread', 'Opening_Odds'])

In [ ]:
columns = ['Betmgm', 'Betmgm_Odds', 'Draft_Kings_Odds', 'Draft_Kings_Odds', 
           'Fanduel_Odds', 'Fanduel_Odds', 'Caesars_Odds', 'Caesars_Odds', 
           'Pointsbet_Odds', 'Pointsbet_Odds', 'Wynn_Odds', 'Wynn_Odds', 
           'Betrivers_Odds', 'Betrivers_Odds']

In [ ]:
df_2 = pd.DataFrame(rows_to_add, columns=columns)

In [ ]:
betting_df = df_1.merge(df_2, left_index=True, right_index=True)
betting_df

In [ ]:
if betting_df.shape[0] == 0:
    print('none')
else:
    print('not none')

In [ ]:
betting_df.shape[0]